In [2]:
import math
import numpy as np
import pandas as pd
import networkx as nx
from functools import partial
from datetime import datetime
from memory_profiler import memory_usage

In [3]:
def euc_2d(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    d = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return d

def generate_nodes(p,data):
    return np.apply_along_axis(lambda x: euc_2d((p[1],p[2]),(x[1],x[2])),1,data)

def generate_graph(archive):
    f = open("./datasets/"+archive[0],"r")
    data = f.readlines()
    data = data[data.index("NODE_COORD_SECTION\n")+1:-1]
    d = np.array(list(map(lambda x: [float(i) for i in x.split()],data)))
    nodes = np.apply_along_axis(lambda x: x,0,d)
    edges = np.apply_along_axis(lambda x: generate_nodes(x,nodes),1,nodes)

    G = nx.Graph()
    #Create Nodes
    G.add_nodes_from(nodes[:,0])

    for i in range(len(nodes[:,0])):
        for j in range(len(nodes[:,0])):
            G.add_weighted_edges_from([(i+1,j+1,edges[i][j])])
    f.close()
    return G

# Twice-Around-The-Tree(TAT)

In [4]:
def sum_edges(G,ord):
    c = 0
    for i in range(len(ord)-1):   
        c += G[ord[i]][ord[i+1]]['weight']
    return c + G[ord[0]][ord[len(ord)-1]]['weight']

def twice_around_the_tree(G):
    T = nx.minimum_spanning_tree(G)
    ordenation = list(nx.dfs_preorder_nodes(T, 1))
    total = sum_edges(G, ordenation)
    return total

# Christofides

In [5]:
def christofides(G):
  T = nx.minimum_spanning_tree(G) 
  n_odd = list(filter(lambda x: (x[1]%2)!=0 ,T.degree))
  min_m = nx.min_weight_matching(G.subgraph(n_odd)) 
  h = nx.MultiGraph(T)
  h.add_edges_from(min_m)
  ordenation = list(nx.dfs_preorder_nodes(h, 1))
  total = sum_edges(G, ordenation)
  return total

# Wrap Up Pipeline

In [6]:
def wrap_up_tat(G):
    start=datetime.now()
    M_TAT, D_TAT =  memory_usage(partial(twice_around_the_tree,G),interval=1,max_usage=True,retval=True)
    T_TAT = datetime.now()-start
    return [T_TAT.total_seconds(),M_TAT,D_TAT]

def wrap_up_chf(G):
    start=datetime.now()
    M_CHF, D_CHF =  memory_usage(partial(christofides,G),interval=1,max_usage=True,retval=True)
    T_CHF = datetime.now()-start
    return [T_CHF.total_seconds(),M_CHF,D_CHF]

In [7]:
file = open('./tp2_datasets.txt', 'r')

archives = []

for line in file.readlines()[1:]:
    name = line.split()[0]+".tsp"
    n_nodes = line.split()[1]
    opt = line.split()[2]
    opt = opt if "[" not in opt else opt[opt.index(",")+1:-1]
    archives.append([name,n_nodes,opt])
file.close()

In [74]:
TAT_DATA = []
for arch in archives:
    G = generate_graph(arch)
    results = wrap_up_tat(G)
    results.insert(0,arch[0])
    results.append(float(arch[2]))
    results.append(int(arch[1]))
    TAT_DATA.append(results)
    print(f"---------------Dataset:{arch[0]} done in {results[1]}----------------")

---------------Dataset:eil51.tsp done in 6.399796----------------
---------------Dataset:berlin52.tsp done in 6.143476----------------
---------------Dataset:st70.tsp done in 6.563385----------------
---------------Dataset:eil76.tsp done in 5.862652----------------
---------------Dataset:pr76.tsp done in 7.207755----------------
---------------Dataset:rat99.tsp done in 4.582917----------------
---------------Dataset:kroA100.tsp done in 7.570693----------------
---------------Dataset:kroB100.tsp done in 6.209035----------------
---------------Dataset:kroC100.tsp done in 5.898284----------------
---------------Dataset:kroD100.tsp done in 5.23298----------------
---------------Dataset:kroE100.tsp done in 7.06383----------------
---------------Dataset:rd100.tsp done in 5.364921----------------
---------------Dataset:eil101.tsp done in 6.842048----------------
---------------Dataset:lin105.tsp done in 5.934035----------------
---------------Dataset:pr107.tsp done in 5.764001----------------

MemoryError: 

In [ ]:
results_tat = pd.DataFrame(TAT_DATA,columns=["Dataset","ExecutionTime","MemoryUsage","FoundWeight","OptimalWeight","NumberNodes"])
results_tat["Quality"] = results_tat["FoundWeight"]/results_tat["OptimalWeight"]
results_tat.to_csv("results_tat.csv",sep=",",header=True,index=False)

In [8]:
CHF_DATA = []
for arch in archives[0:5]:
    G = generate_graph(arch)
    results = wrap_up_chf(G)
    results.insert(0,arch[0])
    results.append(float(arch[2]))
    results.append(int(arch[1]))
    CHF_DATA.append(results)
    print(f"---------------Dataset:{arch[0]} done in {results[1]}----------------")

---------------Dataset:eil51.tsp done in 5.197395----------------
---------------Dataset:berlin52.tsp done in 5.473623----------------
---------------Dataset:st70.tsp done in 6.176966----------------
---------------Dataset:eil76.tsp done in 4.458985----------------
---------------Dataset:pr76.tsp done in 5.493253----------------


In [9]:
results_chf = pd.DataFrame(CHF_DATA,columns=["Dataset","ExecutionTime","MemoryUsage","FoundWeight","OptimalWeight","NumberNodes"])
results_chf["Quality"] = results_chf["FoundWeight"]/results_chf["OptimalWeight"]
results_chf.to_csv("results_chf.csv",sep=",",header=True,index=False)